In [ ]:
# Import library
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split

In [ ]:
# Import data into pandas dataframe
df_pnj=pd.read_csv('/content/PNJ.csv')
df_pnj=pd.DataFrame(df_pnj)

In [ ]:
# Get information from dataset
df_pnj.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 125309 entries, 0 to 125308
Data columns (total 8 columns):
 #   Column         Non-Null Count   Dtype  
---  ------         --------------   -----  
 0   Ticker         125309 non-null  object 
 1   Date/Time      125309 non-null  object 
 2   Open           125309 non-null  float64
 3   High           125309 non-null  float64
 4   Low            125309 non-null  float64
 5   Close          125309 non-null  float64
 6   Volume         125309 non-null  int64  
 7   Open Interest  125309 non-null  int64  
dtypes: float64(4), int64(2), object(2)
memory usage: 7.6+ MB


In [ ]:
df_pnj.describe()

,Open,High,Low,Close,Volume,Open Interest
count,125309.000000,125309.000000,125309.000000,125309.000000,125309.000000,125309.0
mean,72.182052,72.372590,72.042287,72.179371,3031.830451,0.0
std,9.900263,9.995786,9.830847,9.899401,7248.952968,0.0
min,44.430000,44.530000,44.040000,44.140000,1.000000,0.0
25%,64.600000,64.700000,64.570000,64.600000,220.000000,0.0
50%,72.800000,73.010000,72.640000,72.830000,1000.000000,0.0
75%,80.160000,80.350000,80.060000,80.160000,3130.000000,0.0
max,100.160000,100.160000,100.160000,100.160000,580730.000000,0.0


In [ ]:
# Change type of Date/Time column, keep the last price of day and rename
def changeDateTime(df):
    df['Date/Time'] = pd.to_datetime(df['Date/Time'])
    # Change the line to drop the rows and update dataset
    df.drop(df[df['Date/Time'].dt.time != pd.to_datetime('14:46:00').time()].index, inplace=True)
    df.reset_index(drop=True, inplace=True)
    # Convert to date only
    df['Date/Time'] = df['Date/Time'].dt.date
    # Rename the new date column
    df.rename(columns={'Date/Time': 'Date'}, inplace=True)

changeDateTime(df_pnj)

In [ ]:
#Get necessary data
def prepareData(df):
  data = df[['Date', 'Close']]
  dataset=df['Close'].values.reshape(-1,1)
  # Set the training dataset with 85% of given data
  training_data_len=int(np.ceil(len(dataset)*0.85))
  training_data_len
  return data, dataset, training_data_len

In [ ]:
PNJ_data, PNJ_dataset,PNJ_train_len=prepareData(df_pnj)

In [ ]:
# Scale the data
from sklearn.preprocessing import MinMaxScaler
scaler = MinMaxScaler(feature_range=(0,1))
scaled_data = scaler.fit_transform(PNJ_dataset)

In [ ]:
# Create the training data set
def createTrainSet(scaled_data, training_len):
  # Create the scaled training data set
  train_data = scaled_data[0:int(training_len), :]
  # Split the data into x_train and y_train data sets
  x_train = []
  y_train = []

  for i in range(100, len(train_data)):
      x_train.append(train_data[i-100:i, 0])
      y_train.append(train_data[i, 0])

  # Convert the x_train and y_train to numpy arrays
  x_train, y_train = np.array(x_train), np.array(y_train)

  # Reshape the data
  x_train = np.reshape(x_train, (x_train.shape[0], x_train.shape[1], 1))
  return x_train, y_train

In [ ]:
x_train,y_train=createTrainSet(scaled_data,PNJ_train_len)

In [ ]:
from keras.models import Sequential
from keras.layers import Dense, LSTM

# Build the LSTM model
model = Sequential()
model.add(LSTM(128, return_sequences=True, input_shape= (x_train.shape[1], 1)))
model.add(LSTM(64, return_sequences=False))
model.add(Dense(25))
model.add(Dense(1))

# Compile the model
model.compile(optimizer='adam', loss='mean_squared_error')

# Train the model
model.fit(x_train, y_train, batch_size=6, epochs=15)

Epoch 1/15
84/84 [==============================] - 13s 109ms/step - loss: 0.0117
Epoch 2/15
84/84 [==============================] - 8s 100ms/step - loss: 0.0037
Epoch 3/15
84/84 [==============================] - 8s 91ms/step - loss: 0.0037
Epoch 4/15
84/84 [==============================] - 8s 101ms/step - loss: 0.0027
Epoch 5/15
84/84 [==============================] - 7s 86ms/step - loss: 0.0022
Epoch 6/15
84/84 [==============================] - 9s 102ms/step - loss: 0.0020
Epoch 7/15
84/84 [==============================] - 8s 90ms/step - loss: 0.0020
Epoch 8/15
84/84 [==============================] - 10s 123ms/step - loss: 0.0020
Epoch 9/15
84/84 [==============================] - 9s 102ms/step - loss: 0.0017
Epoch 10/15
84/84 [==============================] - 7s 89ms/step - loss: 0.0016
Epoch 11/15
84/84 [==============================] - 16s 186ms/step - loss: 0.0015
Epoch 12/15
84/84 [==============================] - 11s 125ms/step - loss: 0.0015
Epoch 13/15
84/84 [======

In [ ]:
# Create the testing data set
def createTestSet(dataset,scaled_data,training_len):
  # Create a new array containing scaled values
  test_data = scaled_data[training_len - 100: , :]
  # Create the data sets x_test and y_test
  x_test = []
  y_test = dataset[training_len:, :]
  for i in range(100, len(test_data)):
      x_test.append(test_data[i-100:i, 0])

  # Convert the data to a numpy array
  x_test = np.array(x_test)

  # Reshape the data
  x_test = np.reshape(x_test, (x_test.shape[0], x_test.shape[1], 1 ))
  return x_test, y_test

def predictValue(x_test,model,scaler):
  # Get the models predicted price values
  predictions = model.predict(x_test)
  predictions = scaler.inverse_transform(predictions)
  return predictions


In [ ]:
x_test,y_test=createTestSet(PNJ_dataset,scaled_data,PNJ_train_len)
prediction=predictValue(x_test,model,scaler)

4/4 [==============================] - 3s 143ms/step


In [ ]:
#Get the root mean squared error (RMSE)
rmse = np.sqrt(np.mean(((prediction - y_test) ** 2)))
rmse

1.4753164303821127

In [ ]:
# Make validation data
valid = PNJ_data[PNJ_train_len:]
valid['Prediction'] = prediction
# Fill NaN values with 0
valid['Prediction'].fillna(0, inplace=True)

<ipython-input-18-8e031614e24c>:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  valid['Prediction'] = prediction
<ipython-input-18-8e031614e24c>:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  valid['Prediction'].fillna(0, inplace=True)


In [ ]:
import plotly.express as px
import plotly.graph_objects as go

# Create a line plot for 'Close' values
fig = px.line(valid, x="Date", y="Close", title='Close vs Prediction')

# Add 'Prediction' line to the existing figure
fig.add_trace(go.Scatter(x=valid["Date"], y=valid["Prediction"], mode='lines', name='Prediction'))

# Show the plot
fig.show()


In [ ]:
valid.head(10)

,Date,Close,Prediction
599,2020-07-27,52.10,58.128113
600,2020-07-28,54.00,55.432644
601,2020-07-29,51.50,54.396862
602,2020-07-30,51.50,53.147610
603,2020-07-31,51.60,52.405125
604,2020-08-03,52.80,52.154961
605,2020-08-04,53.50,52.638290
606,2020-08-05,54.48,53.440617
607,2020-08-06,55.18,54.438843
608,2020-08-07,54.78,55.393490


In [ ]:
# Calculate the different with the first date
real_diff=valid['Close']-valid['Close'].iloc[0]
pred_diff=valid['Prediction']-valid['Prediction'].iloc[0]

In [ ]:
# Plot the Real Movement vs Prediction Movement
from plotly.subplots import make_subplots
fig=make_subplots(rows=2, cols=1)
fig.add_trace(go.Bar(x=valid['Date'],y=real_diff,name='Real'),row=1,col=1)
fig.add_trace(go.Bar(x=valid['Date'],y=pred_diff,name='Prediction'),row=2,col=1)

fig.update_layout(
    autosize=False,
    width=800,
    height=800,
    title='Real Movement vs Prediction Movement'
)
fig.show()